In [ ]:

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline
import matplotlib.image as mpimg
import splitfolders
import cv2
import glob

In [ ]:
from PIL import Image

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
# FIXING IMAGES EXTENSIONS

for name in glob.glob('/Hacathon_proj/dataset/images/*'):
    if '.JPG' in name:
        os.rename(name,name.replace('JPG','jpg'))


In [ ]:
train_dataset=pd.read_csv("/Hacathon_proj/dataset/train.csv")
test_dataset=pd.read_csv("/Hacathon_proj/dataset/test.csv")
images_directory='/Hacathon_proj/dataset/images/'

In [ ]:
dimensions=train_dataset.iloc[:,3:]
dimensions.describe()

In [ ]:
train_dataset['name'].unique()

In [ ]:
class_dictionary={
    3:'GARBAGE',
    7:'BAD_BILLBOARD',
    8:'SAND_ON_ROAD',
    0:'GRAFFITI',
    2:'POTHOLES',
    9:'CLUTTER_SIDEWALK',
    4:'CONSTRUCTION_ROAD',
    5:'BROKEN_SIGNAGE',
    10:'UNKEPT_FACADE',
    1:'FADED_SIGNAGE',
    6:'BAD_STREETLIGHT'
    
}

In [ ]:
train_directory=images_directory
img_path=[]
count=0
for image in train_dataset['image_path']:
    try:
        img_path.append(train_directory+train_dataset['image_path'][count])
        count+=1
    except:
        print('dfvd')
        count+=1
        continue  

In [ ]:
train_x=[]
images_width=[]
images_height=[]
train_y=[]
count=0
for img in range(0,10000):
    try:
        img=cv2.imread(img_path[count])
        img=cv2.resize(img,(960,540))
        xmin=int(train_dataset['xmin'][count])
        ymin=int(train_dataset['ymin'][count])
        xmax=int(train_dataset['xmax'][count])
        ymax=int(train_dataset['ymax'][count])
        img=img[ymin:ymax, xmin:xmax]
        height,width,rgb=img.shape
        if width >=128 and height >=64:
            train_y.append(train_dataset['class'][count])
            images_width.append(width)
            images_height.append(height)
            img=cv2.resize(img,(128,64))
            train_x.append(img)
        count+=1
    except:
        count+=1
        continue
        

    
    
    

In [ ]:
test_x=[]
test_y=[]
count=10001
for img in range(10001,15000):
    try:
        img=cv2.imread(img_path[count])
        img=cv2.resize(img,(960,540))
        xmin=int(train_dataset['xmin'][count])
        ymin=int(train_dataset['ymin'][count])
        xmax=int(train_dataset['xmax'][count])
        ymax=int(train_dataset['ymax'][count])
        img=img[ymin:ymax, xmin:xmax]
        height,width,rgb=img.shape
        if width >=128 and height >=64:
            test_y.append(train_dataset['class'][count])
            img=cv2.resize(img,(128,64))
            test_x.append(img)
        count+=1
    except:
        count+=1
        continue

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Generator For Augmentation 

train_datagen = ImageDataGenerator(
    rotation_range=15,shear_range=15,width_shift_range=0.05,
          height_shift_range=0.05,zoom_range=0.1
   
    
    )
test_datagen = ImageDataGenerator()

In [ ]:

train_x=np.array(train_x).reshape(-1,128,64,3)
# for img in train_x:
#     img=np.array(img).reshape(-1,128,64,3)
#     train_array.append(img)

train_y = np.array([np.array(val) for val in train_y])

#Normalization 
# train_array=np.array(train_array)
train_x=train_x/255

In [ ]:
test_x=np.array(test_x).reshape(-1,128,64,3)
test_y = np.array([np.array(val) for val in test_y])
test_x=test_x/255

In [ ]:
plt.title(train_y[65])
plt.imshow(train_x[65])

In [ ]:
train_generator=train_datagen.flow(train_x,train_y,batch_size=8)

In [ ]:
import keras
def myModel():
    input = keras.Input(shape= (128,64,3))
    x = keras.layers.Conv2D(64, (3, 3), activation='relu',)(input)
    x = keras.layers.MaxPooling2D((2, 2))(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Conv2D(64, (3, 3), activation='relu',)(x)
    x = keras.layers.MaxPooling2D((2, 2))(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Conv2D(128, (3, 3), activation='relu',)(x)
    x = keras.layers.MaxPooling2D((2, 2))(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation='relu',)(x)
    x = keras.layers.Dropout(0.4)(x)
    out1 = keras.layers.Dense(11, activation='softmax')(x)
    model = keras.models.Model(inputs=input,outputs=[out1])
    return model

In [ ]:
model =myModel()

In [ ]:
with tf.device('/cpu:0'):
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
train_x.__len__()

In [ ]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

In [ ]:
model.fit(train_x,train_y, batch_size = 64, epochs = 50,verbose=1,shuffle=True) 

In [ ]:
# image=cv2.imread(images_directory+train_dataset['image_path'][2400])
# image=cv2.resize(image,(128,128))

In [ ]:
# plt.imshow(image)

In [ ]:
test_loss, test_acc = model.evaluate(test_x,test_y, verbose=1,batch_size=64)
print("The accuracy of the model is:")
print(test_acc)

In [ ]:
img_path.__len__()

In [ ]:
img_path[6070]

In [ ]:
image = cv2.imread(img_path[17560])
    
temp = cv2.resize(image,(960,540))
xmin=int(train_dataset['xmin'][17460])
ymin=int(train_dataset['ymin'][17460])
xmax=int(train_dataset['xmax'][17460])
ymax=int(train_dataset['ymax'][17460])
temp=temp[ymin:ymax, xmin:xmax]
plt.title(train_dataset['name'][17460])
plt.imshow(temp)
temp=cv2.resize(temp,(128,64))
temp = tf.convert_to_tensor(temp)
temp = tf.reshape(temp,(-1,128,64,3))
temp = temp/255
pred= model.predict(temp)

In [ ]:
test = cv2.imread(img_path[17460])
plt.title(train_dataset['name'][17460])
plt.imshow(test)

In [ ]:
class_dictionary[np.argmax(pred)]